<h1>Coffee (Worldwide) - Data Visualisation</h1>

<h2>List of content:</h2>
<ol>
    <li>Used Libraries</li>
    <li>Functions</li>
    <li>Connecting to database</li>
    <li>Loading data</li>
    <li>Creating tables</li>
    <li>Analasys:</li>
    <ul>
        <li>Analysis Goals</li>
        <li>Analising data:</li>
        <ul>
            <li>Domestic Consumption</li>
            <li>Imports</li>
            <li>Retail Prices</li>
            <li>Total Production</li>
        </ul>
    </ul>
 </ol>

<h3>1. Used Libraries</h3>

In [ ]:
import csv
import sqlite3 as sql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<h3>2. Functions</h3>

<h4><b>1) Table Creator</b></h4>
<p>
    &ensp;<b>Description:</b></br> 
    &ensp;Create table based on given name, .csv file,</br> 
    &ensp;CREATE (queryCreate) and INSERT (queryInsert) queries.
</p>
<p>
    &ensp;<b style = "color:red";>NOTE</b></br>
    &ensp;
    If database exist, code won't be executed and will return OperationalError.
    </br>
    &ensp;To make it work, delete file called "coffee.db" from folder.</br>
</p>

In [ ]:
def createTable(file, tableName, queryCreate, queryInsert):
    
    variablesTemp = []  #   Creating list of variables used in queryCreate
                        #   which will contain table name and column names

    variablesTemp.append(tableName)

    with open(file) as f:
        readf = csv.reader(f, delimiter= ';')
        header = next(readf)   #    list of column names 
        [variablesTemp.append(col) for col in header]
        cur.execute(queryCreate % tuple(variablesTemp))  #   Executing query
        rows = [row for row in readf]

        #   Inserting data into table
        for row in rows:
            variablesTemp = []  #   Creating list of variables used in queryInsert
                                #   which will contain table name and data
            variablesTemp.append(tableName)
            [variablesTemp.append(col) for col in row]

            #   Replacing apsotrophe (') to backtiks (`), becouse apostrophe
            #   in variable breaks query

            variablesTemp = [variable.replace("'","`") 
                             for variable in variablesTemp]
            
            cur.execute(queryInsert % tuple(variablesTemp))  #  Executing query
            con.commit()

<h4><b>2) Querry To List</b></h4>
<p>
    &ensp;<b>Description:</b></br>
    &ensp;Creates nested list based on provided table name,</br> 
    &ensp;SELECT query, list of columns (used in query) and filter</br>
    &ensp;(for WHERE condition which is used in query)</br>
</p>

In [ ]:
def querryList(tableName, # name of table 
               query, #    string variable which contains SELECTION querry
               columns = '',    #    list of columns used in querry
               filter = '' #   used for WHERE condition in querry
                ):
    
    variables = []
    #   Adding comprehensive list of column names to variables used in querry
    if columns: #   Adding not empty list of column
        variables = [column for column in columns]

    #   Adding table name to the list of variables
    variables.append(tableName)
    
    if filter:  #   Adding not empty filter
        variables.append(filter)

    #   Executing querry with variables given in function
    res = cur.execute(query% tuple(variables))
    
    #  Writing query to list 
    listData = res.fetchall()

    #   Return nested list
    return listData

<h3>3. Connecting to database</h3>

In [ ]:
con = sql.connect("coffee.db")
cur = con.cursor() #    Creating cursor which will be resposible for
                   #    executing queries

<h3>4. Loading data</h3>

<p>
    Creating dictionary with file names (in .csv format) and table names</br>
    assigned to those files.</br>
</p>

In [ ]:
#   dict = {file_name: table_name}
files = { 
    "retail-prices.csv": "prices",
    "imports.csv": "imports",
    "total-production.csv": "production"
        }

<h4>Data description</h4>
<p>Used files:</p>
<ul>
  <li><b>retail-prices.csv</b> - Retail prices of roasted coffee in
    selected importing countries</li>
  <li><b>imports.csv</b> - Imports by selected importing countries</li>
  <li><b>total-production.csv</b> - Total production by all exporting</br>
    countries</li>
</ul>
<b style = 'color:red'>Note</b>
<p>Files have been modified with use of Microsoft Excel for the purpose of<br>
data visualization</p>
Original dataset: 
<a href=
"https://www.kaggle.com/datasets/yamaerenay/ico-coffee-dataset-worldwide">
  Link
</a>

<h3>5. Creating tables from given files</h3>

In [ ]:
queryCreate = "CREATE TABLE %s (%s VARCHAR, %s VARCHAR, %s FLOAT)"
queryInsert = "INSERT INTO %s VALUES('%s', '%s', '%s')"

for file, tableName in files.items():
    createTable(file, tableName, queryCreate, queryInsert)

<h3>6. Analasys</h3>

<h4>&ensp;Analysis Goals</h4>
<ol>
    <li>Total summary</li>
    <li>Summary of average values over the years</li>
    <li>Summary of chosen country</li>
    <li>Summary of chosen year</li>
</ol>

<p>Used mesurments:</p>
<ul>
    <li>number of records(count),</li> 
    <li>mean value(mean),</li>
    <li> <a href="https://en.wikipedia.org/wiki/Standard_deviation">
    standard deviation(std),</a></li>
    <li>minimum(min) and maximum(max) value,</li>
    <li> <a href="https://en.wikipedia.org/wiki/Quartile">
    first(25%), second(50%) and third(75%) quartile</a></li>
</ul>

<p>Used graphs:</p>
<ul>
    <li>Linear</li>
    <li>Column (Horizontal)</li>
</ul>


<h4>&ensp;Analysing data</h4>
<p>Used queries:</p>


In [ ]:
queryINF = "PRAGMA table_info(%s)"  #   table information
queryAll = "SELECT * FROM %s"   #   all data from table
queryAVG = "SELECT %s, AVG(%s) FROM %s GROUP BY 1"
queryWhereCountry = "SELECT * FROM %s WHERE Country = '%s'"
queryWhereYear = "SELECT * FROM %s WHERE Year = '%s'"

<h5>&emsp;<b>Retail Prices</b></h5>

<p>&emsp;1. Total summary</p>

<p>Creating dataframe and summary</p>

In [ ]:
table = 'prices'

    #   Getting column names from table
data = querryList(table, queryINF)
header = [col[1] for col in data]

    #   Getting table data
data = querryList(table, queryAll)

    #   Data Frame
dataframe = pd.DataFrame(data, columns=header)
dataframe = dataframe.replace('', np.nan)   #   Converting empty string  
                                            #   and 0 to 'Not a Number'
dataframe = dataframe.dropna() # removing records with NaN values
pd.set_option("display.max_rows", 25)
display(dataframe)
    #   Summary 

maxRow = dataframe.loc[dataframe['Value'] == max(dataframe['Value']), 
                        ['Country', 'Year']]
maxRow = maxRow.iloc[0]

minRow = dataframe.loc[dataframe['Value'] == min(dataframe['Value']), 
                        ['Country', 'Year']]
minRow = minRow.iloc[0]

print("SUMMARY")
print("Country with highest value: %s in year: %s"% 
      (maxRow.iloc[0], maxRow.iloc[1]))
print("Country with lowest value: %s in year: %s"% 
      (minRow.iloc[0], minRow.iloc[1]))
dataframe.describe()

<p>&emsp;2. Summary of average values over the years</p>

<p>Creating Dataframe and summary</p>

In [ ]:
cols = ['Year', 'Value']
data = querryList(table, queryAVG, cols)

dataframe = pd.DataFrame(data, columns = ['Year', 'Average Value'])
display(dataframe)
dataframe.describe()

maxAVG = dataframe.loc[dataframe['Average Value'] == 
                       max(dataframe['Average Value']), 'Year']
maxAVG = maxAVG.iloc[0]

minAVG = dataframe.loc[dataframe['Average Value'] == 
                       min(dataframe['Average Value']), 'Year']
minAVG = minAVG.iloc[0]

print("SUMMARY")
print("Year with highest average value: %s"% maxAVG)
print("Year with lowest average value: %s"% minAVG)
dataframe.describe()

<p>Creating graph</p>

In [ ]:
yearsPrice = list(dataframe["Year"])
avgPrice = list(dataframe["Average Value"])

plt.style.use('ggplot')
plt.figure(figsize=(14,5))
plt.plot(yearsPrice, avgPrice, linewidth = 4)

plt.title('Average Retail prices of roasted coffee'
            '\n in selected importing countries over the years 1990 - 2018' 
            '\n (In USD/kg)', fontsize = 18)
plt.xlabel('Year', fontsize = 16)
plt.ylabel('(USD / kg)', fontsize = 16)

plt.xticks([i for i in range(0, 29, 2)], fontsize = 12)
plt.yticks([j for j in range(8, 16, 2)], fontsize = 12)

plt.show()

<p>&emsp;3. Summary of chosen country</p>

<p>Creating Dataframe and summary</p>

In [ ]:
country = input("Insert country name: ")
#country = "Norway" #    Example value
data = querryList(table, queryWhereCountry, filter = country)

dataframe = pd.DataFrame(data, columns = header)
dataframe = dataframe.replace('', np.nan)   #   Converting empty string  
                                            #    and 0 t 'Not a Number'
dataframe = dataframe.dropna() # removing records with NaN values
dataframe = dataframe.drop(columns = 'Country')
print("Chosen country: %s"% country.upper())
display(dataframe)

maxYear = dataframe.loc[dataframe['Value'] == max(dataframe['Value']), 'Year']
maxYear = maxYear.iloc[0]

minYear = dataframe.loc[dataframe['Value'] == min(dataframe['Value']), 'Year']
minYear = minYear.iloc[0]

print("SUMMARY")
print("Year with highest value: %s"% maxYear)
print("Year with lowest value: %s"% minYear)
dataframe.describe()

<p>Creating graph</p>

In [ ]:
yearsCountry = list(dataframe['Year'])
valueCountry = list(dataframe['Value'])

plt.style.use('ggplot')
plt.figure(figsize=(14,5))
plt.plot(yearsCountry, valueCountry, linewidth = 4)

plt.title('Retail prices in %s'
        '\n  over the years 1990 - 2018' 
        '\n (In USD/kg)'% country, fontsize = 18)

plt.xlabel('Year', fontsize = 16)
plt.ylabel('(USD / kg)', fontsize = 16)

plt.xticks([i for i in range(0, 29, 2)], fontsize = 12)
plt.yticks(fontsize = 12)

plt.show()

<p>&emsp;4. Summary of chosen year</p>

<p>Creating Dataframe and summary</p>

In [ ]:
year = input("Insert year: ")
#year = "2018" #    Example value
data = querryList(table, queryWhereYear, filter = year)

dataframe = pd.DataFrame(data, columns = header)
dataframe = dataframe.replace('-', np.nan)   #   Converting empty string to  
                                            #   Not a Number
dataframe = dataframe.dropna() # removing records with NaN values
dataframe = dataframe.drop(columns = 'Year')
print("Chosen year: %s"% year)
display(dataframe)

maxCountry = dataframe.loc[dataframe['Value'] 
                        == max(dataframe['Value']), 'Country']
maxCountry = maxCountry.iloc[0]

minCountry = dataframe.loc[dataframe['Value'] 
                        == min(dataframe['Value']), 'Country']
minCountry = minCountry.iloc[0]

print("SUMMARY")
print("Country with highest value: %s"% maxCountry)
print("Country with lowest value: %s"% minCountry)
dataframe.describe()

<p>Creating graph</p>

In [ ]:
dataframe = dataframe.sort_values(by = 'Value', axis = 0)
countriesYear = list(dataframe['Country'])
valueYear = list(dataframe['Value'])

plt.style.use('ggplot')
plt.figure(figsize=(14,5))
plt.barh(countriesYear, valueYear)
plt.title("Retail prices of roasted coffee in year %s (In USD/kg)"% year)
plt.ylabel("Country")
plt.xlabel("Value (USD/kg)")

plt.show()

<h5>&emsp;<b>Imports</b></h5>

<p>&emsp;1. Total summary</p>

<p>Creating dataframe and summary</p>


In [ ]:
table = 'imports'

    #   Getting column names from table
data = querryList(table, queryINF)
header = [col[1] for col in data]

    #   Getting table data
data = querryList(table, queryAll)

    #   Data Frame
dataframe = pd.DataFrame(data, columns=header)
dataframe = dataframe.replace('', np.nan)   #   Converting empty string to  
                                            #   Not a Number
dataframe = dataframe.dropna() # removing records with NaN values
pd.set_option("display.max_rows", 25)
display(dataframe)
    #   Summary 

maxRow = dataframe.loc[dataframe['Value'] == max(dataframe['Value']), 
                        ['Country', 'Year']]
maxRow = maxRow.iloc[0]

minRow = dataframe.loc[dataframe['Value'] == min(dataframe['Value']), 
                        ['Country', 'Year']]
minRow = minRow.iloc[0]

print("SUMMARY")
print("Country with highest value: '%s' in year: '%s'"% 
      (maxRow.iloc[0], maxRow.iloc[1]))
print("Country with lowest value: '%s' in year: '%s'"% 
      (minRow.iloc[0], minRow.iloc[1]))
dataframe.describe()

<p>&emsp;2. Summary of average values over the years</p>

<p>Creating Dataframe and summary</p>


In [ ]:
cols = ['Year', 'Value']
data = querryList(table, queryAVG, cols)

dataframe = pd.DataFrame(data, columns = ['Year', 'Average Value'])
display(dataframe)
dataframe.describe()

maxAVG = dataframe.loc[dataframe['Average Value'] == 
                       max(dataframe['Average Value']), 'Year']
maxAVG = maxAVG.iloc[0]

minAVG = dataframe.loc[dataframe['Average Value'] == 
                       min(dataframe['Average Value']), 'Year']
minAVG = minAVG.iloc[0]

print("SUMMARY")
print("Year with highest average value: %s"% maxAVG)
print("Year with lowest average value: %s"% minAVG)
dataframe.describe()

<p>Creating graph</p>


In [ ]:
yearsPrice = list(dataframe["Year"])
avgPrice = list(dataframe["Average Value"])

plt.style.use('ggplot')
plt.figure(figsize=(14,5))
plt.plot(yearsPrice, avgPrice, linewidth = 4)

plt.title('Average imports of roasted coffee'
            '\n in selected importing countries over the years 1990 - 2018' 
            '\n (In thousand 60-kg bags)', fontsize = 18)
plt.xlabel('Year', fontsize = 16)
plt.ylabel('(USD / kg)', fontsize = 16)

plt.xticks([i for i in range(0, 29, 2)], fontsize = 12)
plt.yticks(fontsize = 12)

plt.show()

<p>&emsp;3. Summary of chosen country</p>

<p> Creating Dataframe and summary</p>


In [ ]:
country = input("Insert country name: ")
#country = "Austria" #    Example value
data = querryList(table, queryWhereCountry, filter = country)

dataframe = pd.DataFrame(data, columns = header)
dataframe = dataframe.replace(['',0], np.nan)   #   Converting empty string to  
                                            #   Not a Number
dataframe = dataframe.dropna() # removing records with NaN values
dataframe = dataframe.drop(columns = 'Country')
print("Chosen country: %s"% country.upper())
display(dataframe)

maxYear = dataframe.loc[dataframe['Value'] == max(dataframe['Value']), 'Year']
maxYear = maxYear.iloc[0]

minYear = dataframe.loc[dataframe['Value'] == min(dataframe['Value']), 'Year']
minYear = minYear.iloc[0]

print("SUMMARY")
print("Year with highest value: %s"% maxYear)
print("Year with lowest value: %s"% minYear)
dataframe.describe()

<p>Creating graph</p>


In [ ]:
yearsCountry = list(dataframe["Year"])
valueCountry = list(dataframe["Value"])

plt.style.use('ggplot')
plt.figure(figsize=(14,5))
plt.plot(yearsCountry, valueCountry, linewidth = 4)

plt.title('Imports in %s'
        '\n  over the years 1990 - 2018' 
        '\n (In thousand 60-kg bags)'% country, fontsize = 18)

plt.xlabel('Year', fontsize = 16)
plt.ylabel('(USD / kg)', fontsize = 16)

plt.xticks([i for i in range(0, 29, 2)], fontsize = 12)
plt.yticks(fontsize = 12)

plt.show()

<p>&emsp;4. Summary of chosen year</p>

<p>Creating Dataframe and summary</p>


In [ ]:
year = input("Insert year: ")
#year = "2010" #    Example value
data = querryList(table, queryWhereYear, filter = year)

dataframe = pd.DataFrame(data, columns = header)
dataframe = dataframe.replace(['', 0], np.nan)   # Converting empty string  
                                            # and 0 to 'Not a Number' value
dataframe = dataframe.dropna() # removing records with NaN values
dataframe = dataframe.drop(columns = 'Year')
print("Chosen year: %s"% year)
display(dataframe)
dataframe.describe()

maxCountry = dataframe.loc[dataframe['Value'] 
                        == max(dataframe['Value']), 'Country']
maxCountry = maxCountry.iloc[0]

minCountry = dataframe.loc[dataframe['Value'] 
                        == min(dataframe['Value']), 'Country']
minCountry = minCountry.iloc[0]

print("SUMMARY")
print("Country with highest value: %s"% maxCountry)
print("Country with lowest value: %s"% minCountry)
dataframe.describe()

<p>Creating graph</p>

In [ ]:
dataframe = dataframe.sort_values(by = 'Value', axis = 0)
countriesYear = list(dataframe['Country'])
valueYear = list(dataframe['Value'])

plt.style.use('ggplot')
plt.figure(figsize=(14,11))
plt.barh(countriesYear, valueYear)
plt.title("Imports of roasted coffee in year %s (In USD/kg)"% year)
plt.ylabel("Country")
plt.xlabel("Value (In thousand 60-kg bags)")

plt.show()

<h5>&emsp;<b>Total production</b></h5>

<p>&emsp;1. Total summary</p>

<p>Creating dataframe and summary</p>


In [ ]:
table = 'production'

    #   Getting column names from table
data = querryList(table, queryINF)
header = [col[1] for col in data]

    #   Getting table data
data = querryList(table, queryAll)

    #   Data Frame
dataframe = pd.DataFrame(data, columns=header)
dataframe = dataframe.replace('', np.nan)   #   Converting empty string  
                                            # and 0 to 'Not a Number'
dataframe = dataframe.dropna() # removing records with NaN values
pd.set_option("display.max_rows", 25)
display(dataframe)
    #   Summary 

maxRow = dataframe.loc[dataframe['Value'] == max(dataframe['Value']), 
                        ['Country', 'Year']]
maxRow = maxRow.iloc[0]

minRow = dataframe.loc[dataframe['Value'] == min(dataframe['Value']), 
                        ['Country', 'Year']]
minRow = minRow.iloc[0]

print("SUMMARY")
print("Country with highest value: '%s' in year: '%s'"% 
      (maxRow.iloc[0], maxRow.iloc[1]))
print("Country with lowest value: '%s' in year: '%s'"% 
      (minRow.iloc[0], minRow.iloc[1]))
dataframe.describe()

<p>&emsp;2. Summary of average values over the years</p>

<p>Creating Dataframe and summary</p>


In [ ]:
cols = ['Year', 'Value']
data = querryList(table, queryAVG, cols)

dataframe = pd.DataFrame(data, columns = ['Year', 'Average Value'])
display(dataframe)
dataframe.describe()

maxAVG = dataframe.loc[dataframe['Average Value'] == 
                       max(dataframe['Average Value']), 'Year']
maxAVG = maxAVG.iloc[0]

minAVG = dataframe.loc[dataframe['Average Value'] == 
                       min(dataframe['Average Value']), 'Year']
minAVG = minAVG.iloc[0]

print("SUMMARY")
print("Year with highest average value: %s"% maxAVG)
print("Year with lowest average value: %s"% minAVG)
dataframe.describe()

<p>Creating graph</p>

In [ ]:
yearsPrice = list(dataframe["Year"])
avgPrice = list(dataframe["Average Value"])

plt.style.use('ggplot')
plt.figure(figsize=(14,5))
plt.plot(yearsPrice, avgPrice, linewidth = 4)

plt.title('Average production of roasted coffee'
            '\n in selected importing countries over the years 1990 - 2018' 
            '\n (In thousand 60-kg bags)', fontsize = 18)
plt.xlabel('Year', fontsize = 16)
plt.ylabel('(USD / kg)', fontsize = 16)

plt.xticks([i for i in range(0, 29, 2)], fontsize = 12)
plt.yticks(fontsize = 12)

plt.show()

<p>&emsp;3. Summary of chosen country</p>

<p>Creating Dataframe and summary</p>

In [ ]:
country = input("Insert country name: ")
#country = "Angola" #    Example value
data = querryList(table, queryWhereCountry, filter = country)

dataframe = pd.DataFrame(data, columns = header)
dataframe = dataframe.replace(['',0], np.nan)   #   Converting empty string  
                                            #  and 0 to 'Not a Number'
dataframe = dataframe.dropna() # removing records with NaN values
dataframe = dataframe.drop(columns = 'Country')
print("Chosen country: %s"% country.upper())
display(dataframe)

maxYear = dataframe.loc[dataframe['Value'] == max(dataframe['Value']), 'Year']
maxYear = maxYear.iloc[0]

minYear = dataframe.loc[dataframe['Value'] == min(dataframe['Value']), 'Year']
minYear = minYear.iloc[0]

print("SUMMARY")
print("Year with highest value: %s"% maxYear)
print("Year with lowest value: %s"% minYear)
dataframe.describe()

<p>Creating graph</p>

In [ ]:
yearsCountry = list(dataframe["Year"])
valueCountry = list(dataframe["Value"])

plt.style.use('ggplot')
plt.figure(figsize=(14,5))
plt.plot(yearsCountry, valueCountry, linewidth = 4)

plt.title('Production in %s'
        '\n  over the years 1990 - 2018' 
        '\n (In thousand 60-kg bags)'% country, fontsize = 18)

plt.xlabel('Year', fontsize = 16)
plt.ylabel('(USD / kg)', fontsize = 16)

plt.xticks([i for i in range(0, 29, 2)], fontsize = 12)
plt.yticks(fontsize = 12)

plt.show()

<p>&emsp;4. Summary of chosen year</p>

<p>Creating Dataframe and summary</p>

In [ ]:
year = input("Insert year: ")
#year = "2018" #    Example value
data = querryList(table, queryWhereYear, filter = year)

dataframe = pd.DataFrame(data, columns = header)
dataframe = dataframe.replace(['', 0], np.nan)   #   Converting empty string  
                                            # an 0 to 'Not a Number'
dataframe = dataframe.dropna() # removing records with NaN values
dataframe = dataframe.drop(columns = 'Year')
print("Chosen year: %s"% year)
display(dataframe)
dataframe.describe()

maxCountry = dataframe.loc[dataframe['Value'] 
                        == max(dataframe['Value']), 'Country']
maxCountry = maxCountry.iloc[0]

minCountry = dataframe.loc[dataframe['Value'] 
                        == min(dataframe['Value']), 'Country']
minCountry = minCountry.iloc[0]

print("SUMMARY")
print("Country with highest value: %s"% maxCountry)
print("Country with lowest value: %s"% minCountry)
dataframe.describe()


<p>Creating graph</p>


In [ ]:
dataframe = dataframe.sort_values(by = 'Value', axis = 0)
countriesYear = list(dataframe['Country'])
valueYear = list(dataframe['Value'])

plt.style.use('ggplot')
plt.figure(figsize=(14,12))
plt.barh(countriesYear, valueYear)
plt.title("Production of roasted coffee in year %s (In USD/kg)"% year)
plt.ylabel("Country")
plt.xlabel("Value (In thousand 60-kg bags)")

plt.show()